In [27]:
import pickle, pandas as pd
from fuzzywuzzy import fuzz
from urllib.parse import urlparse

def get_max_token_ratio(dict_row):
    max_token_set_ratio = 0
    for token in dict_row['website'].split('.'):
        max_token_set_ratio = max(max_token_set_ratio, fuzz.token_set_ratio(token, dict_row['name'].lower()))
    return max_token_set_ratio

with open('list_of_row_dicts.pkl', 'rb') as f:
    list_of_row_dicts = pickle.load(f)

page_rank_lookup = {}
d = pd.read_csv('d.csv')
for _, r in d.iterrows():
    page_rank_lookup[r['domain']] = float(str(r['pg_rank']))
master_list = []
for i in range(0, len(list_of_row_dicts)):
    row_dict = list_of_row_dicts[i]
    row_dict['website'] = row_dict['website'].strip().lower()
    if row_dict['website'] is '':
        parsed_uri = urlparse(row_dict['students_careers_url'])
        row_dict['website'] = '{uri.netloc}'.format(uri=parsed_uri).strip()
        if row_dict['website'] == '':
            continue
        if get_max_token_ratio(row_dict) > 90:
            if row_dict['website'] in page_rank_lookup.keys():
                row_dict['pg_rank'] = page_rank_lookup[row_dict['website']]
                master_list.append(row_dict)
    else:
        if row_dict['website'] in page_rank_lookup.keys():
            row_dict['pg_rank'] = page_rank_lookup[row_dict['website']]
            master_list.append(row_dict)
final_df = pd.DataFrame(master_list)
final_df.to_csv('final_companies_data.csv')